(chpt:theory)=
# Theory

STUB

fdsfsf

updated

3.03pm
PLUS EXCLUDE

In [ ]:
import epsproc as ep

# Set specific LM coeffs by list with setBLMs, items are [l,m,value]
from epsproc.sphCalc import setBLMs

BLM = setBLMs([[0,0,1],[1,1,1],[2,2,1]])   # Note different index
# BLM

# Set the backend to 'pl' for an interactive surface plot with Plotly
ep.sphFromBLMPlot(BLM, facetDim='t', plotFlag = True, backend = 'pl');

## DRAFT NOTES

% From MFRECON manuscript

% TODO: tidy up & add some computational bits directly

### Photoelectron flux in the LF and MF

A (generally) more complicated example is the topic of this article,
i.e. the determination of MF photoelectron distributions, and the
intrinsic molecular photoionization dynamics which underlie the
observables. The former may involve direct or indirect techniques; the
latter is usually defined as the retrieval of the (complex valued)
ionization dipole matrix elements, hence is indirect by definition
(since phase information may not be directly measured). The observables
of interest - the photoelectron flux as a function of energy, ejection
angle, and time - can be written generally as an expansion in spherical
harmonics:

$$\bar{I}(\epsilon,t,\theta,\phi)=\sum_{L=0}^{2n}\sum_{M=-L}^{L}\bar{\beta}_{L,M}(\epsilon,t)Y_{L,M}(\theta,\phi)\label{eq:AF-PAD-general}$$

Here the flux in the laboratory frame (LF) or aligned frame (AF) is
denoted $\bar{I}(\epsilon,t,\theta,\phi)$, with the bar signifying
ensemble averaging, and the molecular frame flux by
$I(\epsilon,t,\theta,\phi)$. Similarly, the expansion parameters
$\bar{\beta}_{L,M}(\epsilon,t)$ include a bar for the LF/AF case. These
observables are generally termed photoelectron angular distributions
(PADs), often with a prefix denoting the reference frame, e.g. LFPADs,
MFPADs, and the associated expansion parameters
$\bar{\beta}_{L,M}(\epsilon,t)$ are generically termed "anisotropy\"
parameters. The polar coordinate system $(\theta,\phi)$ is referenced to
an experimentally-defined axis in the LF/AF case (usually defined by the
laser polarization), and the molecular symmetry axis in the MF (see Fig.
[1](#781808){reference-type="ref" reference="781808"}). The spherical
harmonic rank and order $(L,M)$ are constrained by experimental factors
in the LF/AF, and $n$ is effectively limited by the molecular alignment
(which is correlated with the photon-order for gas phase experiments, or
conservation of angular momentum in the LF more generally [@Yang1948]),
but in the MF is defined by the maximum continuum angular momentum
$n=l_{max}$ imparted by the scattering event [@Dill1976].

For basic cases these limits may be low: for instance, a simple 1-photon
photoionization event ($n=1$) from an isotropic ensemble (zero net
ensemble angular momentum) defines $L_{max}=2$; for cylindrically
symmetric cases (i.e. $D_{\infty h}$ symmetry) $M=0$ only. For MF cases,
$l_{max}=4$ is often given as a reasonable rule-of-thumb for the
continuum - hence $L_{max}=8$ - although in practice higher-$l$ may be
populated. For a realistic example see Sect.
[\[sec:bootstrapping\]](#sec:bootstrapping){reference-type="ref"
reference="sec:bootstrapping"}, and ref. [@hockett2018QMP1] for more
discussion and complex examples.

A broad conceptual overview for a time-dependent measurement scheme and
associated observables is illustrated in Fig.
[1](#781808){reference-type="ref" reference="781808"}; although the
figure shows a specific experimental scheme, the concepts are general.
Note, in particular that the LF measurements involve averaging over an
ensemble of molecules with different orientations, leading to averaging
over the molecular frame observables, which are highly structured.
Compare, in particular, the full MFPADs ($I(\theta,\phi)$ shown in the
bottom left panel, with the relatively unstructured - but
time-dependent - LFPADs ($\bar{I}(\theta,t)$) shown in the bottom right
panel. In this particular example, each time-dependent measurement
involves averaging over a different alignment of the ensemble, and
represent a rich dataset for retrieving matrix elements and
reconstruction of the MFPADs. In cases - such as this example - where
the time-dependence is purely geometric, and is separable from the
ionization matrix elements, the total information content can be broadly
viewed as the number of sets of $\{L,M\}$ at a given $\epsilon$.

## Photoionization dynamics[\[sec:dynamics-intro\]]{#sec:dynamics-intro label="sec:dynamics-intro"}

The core physics of photoionization has been covered extensively in the
literature, and only a very brief overview is provided here with
sufficient detail to introduce the MF reconstruction problem; the reader
is referred to the literature listed in Appendix
[\[sec:theory-lit\]](#sec:theory-lit){reference-type="ref"
reference="sec:theory-lit"} for further details and general discussion.
Technical details of the formalism applied for the reconstruction
techniques discussed herein can be found in Sect.
[\[sec:tensor-formulation\]](#sec:tensor-formulation){reference-type="ref"
reference="sec:tensor-formulation"}.

Photoionization can be described by the coupling of an initial state of
the system to a particular final state (photoion(s) plus free
photoelectron(s)), coupled by an electric field/photon. Very
generically, this can be written as a matrix element
$\langle\Psi_i|\hat{\Gamma}(\boldsymbol{\mathbf{E}})|\Psi_f\rangle$,
where $\hat{\Gamma}(\boldsymbol{\mathbf{E}})$ defines the light-matter
coupling operator (depending on the electric field
$\boldsymbol{\mathbf{E}}$), and $\Psi_i$, $\Psi_f$ the total
wavefunctions of the initial and final states respectively.

There are many flavours of this fundamental light-matter interaction,
depending on system and coupling; the discussion here is confined to the
simplest case of single-photon absorption, in the weak field (or
purturbative), dipolar regime, resulting in a single photoelectron. (For
more discussion of various approximations in photoionzation, see refs.
[@Seideman2002; @Seideman2001].)

Underlying the photoelecton observables is the photoelectron continuum
state $\left|\mathbf{k}\right>$, prepared via photoionization. The
photoelectron momentum vector is denoted generally by
$\boldsymbol{\mathbf{k}}=k\mathbf{\hat{k}}$, in the MF. The ionization
matrix elements associated with this transition provide the set of
quantum amplitudes completely defining the final continuum scattering
state,
$$\left|\Psi_f\right> = \sum{\int{\left|\Psi_{+};\bf{k}\right>\left<\Psi_{+};\mathbf{k}|\Psi_f\right> d\bf{k}}},
\label{eq:cstate}$$ where the sum is over states of the molecular ion
$\left|\Psi_{+}\right>$. The number of ionic states accessed depends on
the nature of the ionizing pulse and interaction. For the dipolar case,

$$\hat{\Gamma}(\boldsymbol{\mathbf{E}}) = \hat{\mathbf{\mu}}.\boldsymbol{\mathbf{E}}$$

Hence,

$$\left<\Psi_{+};\mathbf{k}|\Psi_f\right> =\langle\Psi_{+};\,\mathbf{k}|\hat{\mathbf{\mu}}.\boldsymbol{\mathbf{E}}|\Psi_{i}\rangle
\label{eq:matE-dipole}$$

Where the notation implies a perturbative photoionization event from an
initial state $i$ to a particular ion plus electron state following
absorption of a photon $h\nu$,
$|\Psi_{i}\rangle+h\nu{\rightarrow}|\Psi_{+};\boldsymbol{\mathbf{k}}\rangle$,
and $\hat{\mu}.\boldsymbol{\mathbf{E}}$ is the usual dipole interaction
term [@qOptics], which includes a sum over all electrons $s$ defined in
position space as $\mathbf{r_{s}}$:

$$\hat{\mu}=-e\sum_{s}\mathbf{r_{s}}
\label{eq:dipole-operator}$$

The position space photoelectron wavefunction is typically expressed in
the "partial waves\" basis, expanded as (asymptotic) continuum
eignstates of orbital angular momentum, with angular momentum components
$(l,m)$ (note lower case notation for the partial wave components),

$$\Psi_\mathbf{k}(\bm{r})\equiv\left<\bm{r}|\mathbf{k}\right> = \sum_{lm}Y_{lm}(\mathbf{\hat{k}})\psi_{lm}(\bm{r},k)
\label{eq:elwf}$$

where $\bm{r}$ are MF electronic coordinates and
$Y_{lm}(\mathbf{\hat{k}})$ are the spherical harmonics.

Similarly, the ionization dipole matrix elements can be separated
generally into radial (energy-dependent or 'dynamical' terms) and
geometric (angular momentum) parts (this separation is essentially the
Wigner-Eckart Theorem, see Ref. [@zareAngMom] for general discussion),
and written generally as (using notation similar to [@Reid1991]):

$$\langle\Psi_{+};\,\mathbf{k}|\hat{\mathbf{\mu}}.\boldsymbol{\mathbf{E}}|\Psi_{i}\rangle = \sum_{lm}\gamma_{l,m}\mathbf{r}_{k,l,m}
\label{eq:r-kllam}$$

Provided that the geometric part of the matrix elements $\gamma_{l,m}$ -
which includes the geometric rotations into the LF arising from the dot
product in Eq. [\[eq:r-kllam\]](#eq:r-kllam){reference-type="ref"
reference="eq:r-kllam"} and other angular-momentum coupling terms - are
know, knowledge of the so-called radial (or reduced) dipole matrix
elements, at a given $k$ thus equates to a full description of the
system dynamics (and, hence, the observables).

For the simplest treatment, the radial matrix element can be
approximated as a 1-electron integral involving the initial electronic
state (orbital), and final continuum photoelectron wavefunction:

$$\mathbf{r}_{k,l,m}=\int\psi_{lm}(\bm{r},k)\bm{r}\Psi_{i}(\bm{r})d\bm{r}
\label{eq:r-kllam-integral}$$

As noted above, the geometric terms $\gamma_{l,m}$ are analytical
functions which can be computed for a given case - minimally requiring
knowledge of the molecular symmetry and polarization geometry, although
other factors may also play a role (see Sect.
[\[sec:full-tensor-expansion\]](#sec:full-tensor-expansion){reference-type="ref"
reference="sec:full-tensor-expansion"} for details).

The photoelectron angular distribution (PAD) at a given $(\epsilon,t)$
can then be determined by the squared projection of
$\left|\Psi_f\right>$ onto a specific state
$\left|\Psi_{+};\bf{k}\right>$ (see Sect.
[\[sec:theoretical-techniques\]](#sec:theoretical-techniques){reference-type="ref"
reference="sec:theoretical-techniques"}), and therefore the amplitudes
in Eq. [\[eq:r-kllam\]](#eq:r-kllam){reference-type="ref"
reference="eq:r-kllam"} also determine the observable anisotropy
parameters $\beta_{L,M}(\epsilon,t)$ (Eqn.
[\[eq:AF-PAD-general\]](#eq:AF-PAD-general){reference-type="ref"
reference="eq:AF-PAD-general"}). (Note that the photoelectron energy
$\epsilon$ and momentum $k$ are used somewhat interchangeably herein,
with the former usually preferred in reference to observables.) Note,
also, that in the treatment above there is no time-dependence
incorporated in the notation; however, a time-dependent treatment
readily follows, and may be incorporated either as explicit
time-dependent modulations in the expansion of the wavefunctions for a
given case, or implicitly in the radial matrix elements. Examples of the
former include, e.g. a rotational or vibrational wavepacket, or a
time-dependent laser field. The rotational wavepacket case is discussed
herein (see Sect.
[\[sec:full-tensor-expansion\]](#sec:full-tensor-expansion){reference-type="ref"
reference="sec:full-tensor-expansion"}). The radial matrix elements are
a sensitive function of molecular geometry and electronic configuration
in general, hence may be considered to be responsive to molecular
dynamics, although they are formally time-independent in a
Born-Oppenheimer basis - for further general discussion and examples see
Ref. [@wu2011TimeresolvedPhotoelectronSpectroscopy]; discussions of more
complex cases with electronic and nuclear dynamics can be found in Refs.
[@arasaki2000ProbingWavepacketDynamics; @Seideman2001; @Suzuki2001; @Stolow2008].

Typically, for reconstruction experiments, a given measurement will be
selected to simplify this as much as possible by, e.g., populating only
a single ionic state (or states for which the corresponding observables
are experimentally energetically-resolvable), and with a bandwidth
$d\bf{k}$ which is small enough such that the matrix elements can be
assumed constant. Importantly, the angle-resolved observables are
sensitive to the magnitudes and (relative) phases of these matrix
elements, and can be considered as angular interferograms (Fig.
[1](#781808){reference-type="ref" reference="781808"} top right).


### Tensor formulation of photoionization[\[sec:tensor-formulation\]]{#sec:tensor-formulation label="sec:tensor-formulation"}

A number of authors have treated MFPADs and related problems, see
Appendix [\[sec:theory-lit\]](#sec:theory-lit){reference-type="ref"
reference="sec:theory-lit"} for some examples. Herein, a geometric
tensor based formalism is developed, which is close in spirit to the
treatments given by Underwood and co-workers
[@Reid2000; @Stolow2008; @Underwood2000], but further separates various
sets of physical parameters into dedicated tensors; this allows for a
unified theoretical and numerical treatment, where the latter computes
properties as tensor variables which can be further manipulated and
investigated. Furthermore, the tensors can readily be converted to a
density matrix representation [@BlumDensityMat; @zareAngMom], which is
more natural for some quantities, and also emphasizes the link to
quantum state tomography and other quantum information techniques. Much
of the theoretical background, as well as application to aspects of the
current problem, can be found in the textbooks of Blum [@BlumDensityMat]
and Zare [@zareAngMom].

Within this treatment, the observables can be defined in a series of
simplified forms, emphasizing the quantities of interest for a given
problem. Some details are defined in the following subsections, and
further detailed in Appendix
[\[appendix:formalism\]](#appendix:formalism){reference-type="ref"
reference="appendix:formalism"}.

### Channel functions[\[sec:channel-funcs\]]{#sec:channel-funcs label="sec:channel-funcs"}

A simple form of the equations, amenable to fitting, is to write the
observables in terms of "channel functions\", which define the
ionization continuum for a given case and set of parameters $u$ (e.g.
defined for the MF, or defined for a specific experimental
configuration),

$$\beta_{L,M}^{u}=\sum_{\zeta,\zeta'}\varUpsilon_{L,M}^{u,\zeta\zeta'}\mathbb{I}^{\zeta\zeta'}\label{eqn:channel-fns}$$

Where $\zeta,\zeta'$ collect all the required quantum numbers, and
define all (coherent) pairs of components. The term
$\mathbb{I}^{\zeta\zeta'}$ denotes the coherent square of the ionization
matrix elements:

$$\mathbb{I}^{\zeta,\zeta}=I^{\zeta}(\epsilon)I^{\zeta'*}(\epsilon)
\label{eqn:I-zeta}$$

This is effectively a convolution equation (cf. refs.
[@Reid2000; @gregory2021MolecularFramePhotoelectron]) with channel
functions, for a given "experiment" $u$, summed over all terms
$\zeta,\zeta'$. Aside from the change in notation (which is here chosen
to match the formalism of Refs.
[@Gianturco1994; @Lucchese1986; @Natalense1999]), see also Sect.
[\[sec:mat-ele-conventions\]](#sec:mat-ele-conventions){reference-type="ref"
reference="sec:mat-ele-conventions"}), these matrix elements are
essentially identical to the simplified (radial) forms
$\mathbf{r}_{k,l,m}$ defined in Eqn.
[\[eq:r-kllam\]](#eq:r-kllam){reference-type="ref"
reference="eq:r-kllam"}, in the case where $\zeta=k,l,m$. These complex
matrix elements can also be equivalently defined in a magnitude, phase
form:

$$I^{\zeta}(\epsilon)\equiv\mathbf{r}_{\zeta}\equiv r_{\zeta}e^{i\phi_{\zeta}}$$

This tensorial form is numerically implemented in the ePSproc codebase
[@ePSprocGithub], and is in contradistinction to standard numerical
routines in which the requisite terms are usually computed from
vectorial and/or nested summations, which can be somewhat opaque to
detailed interpretation, and typically implement the full computation of
the observables in one computational routine. The PEMtk codebase
[@hockett2021PEMtkGithub] implements matrix element retrieval based on
this formalism, with pre-computation of all the geometric tensor
components (channel functions) prior to a fitting protocol for matrix
element analysis, essentially a fit to Eqn.
[\[eqn:channel-fns\]](#eqn:channel-fns){reference-type="ref"
reference="eqn:channel-fns"}, with terms $I^{\zeta}(\epsilon)$ as the
unknowns (in magnitude, phase form). The main computational cost of a
tensor-based approach is that more RAM is required to store the full set
of tensor variables.

### Density matrix representation[\[sec:density-mat-basic\]]{#sec:density-mat-basic label="sec:density-mat-basic"}

The density operator associated with the continuum state in
Eq. [\[eq:cstate\]](#eq:cstate){reference-type="ref"
reference="eq:cstate"} is easily written as
$\hat{\rho}=|\Psi_c\rangle\langle\Psi_c|$. In the channel function
basis, this leads to a density matrix given by the radial matrix
elements:

$$\mathbf{\rho}^{\zeta\zeta'} = \mathbb{I}^{\zeta,\zeta'}
\label{eqn:radial-density-mat}$$

Since the matrix elements characterise the scattering event, the density
matrix provides an equivalent characterisation of the scattering event.
An example case is discussed in Sect.
[\[sec:den-mat-N2\]](#sec:den-mat-N2){reference-type="ref"
reference="sec:den-mat-N2"} (see Fig. [11](#998904){reference-type="ref"
reference="998904"}); for more details, and further discussion, see
Sect.
[\[sec:density-mat-full\]](#sec:density-mat-full){reference-type="ref"
reference="sec:density-mat-full"}. Further discussion can also be found
in the literature, see, e.g., Ref. [@BlumDensityMat] for general
discussion, Ref. [@Reid1991] for application in pump-probe schemes.

### Full tensor expansion[\[sec:full-tensor-expansion\]]{#sec:full-tensor-expansion label="sec:full-tensor-expansion"}

In more detail, the channel functions can be given as a set of tensors,
defining each aspect of the problem.

For the MF:

$$\begin{aligned}
\beta_{L,-M}^{\mu_{i},\mu_{f}}(\epsilon) & = & (-1)^{M}\sum_{P,R',R}(2P+1)^{\frac{1}{2}}{E_{P-R}(\hat{e};\mu_{0})}\\
 & \times &\sum_{l,m,\mu}\sum_{l',m',\mu'}(-1)^{(\mu'-\mu_{0})}{\Lambda_{R',R}(R_{\hat{n}};\mu,P,R,R')B_{L,-M}(l,l',m,m')}\\
 & \times & I_{l,m,\mu}^{p_{i}\mu_{i},p_{f}\mu_{f}}(\epsilon)I_{l',m',\mu'}^{p_{i}\mu_{i},p_{f}\mu_{f}*}(\epsilon)\label{eq:BLM-tensor-MF}\end{aligned}$$

And the LF/AF as:

$$\begin{aligned}
\bar{\beta}_{L,-M}^{\mu_{i},\mu_{f}}(E,t) & = & (-1)^{M}\sum_{P,R',R}{[P]^{\frac{1}{2}}}{E_{P-R}(\hat{e};\mu_{0})}\\
 & \times &\sum_{l,m,\mu}\sum_{l',m',\mu'}(-1)^{(\mu'-\mu_{0})}{\Lambda_{R'}(\mu,P,R')B_{L,S-R'}(l,l',m,m')}\\
 & \times &I_{l,m,\mu}^{p_{i}\mu_{i},p_{f}\mu_{f}}(\epsilon)I_{l',m',\mu'}^{p_{i}\mu_{i},p_{f}\mu_{f}*}(\epsilon)\sum_{K,Q,S}\Delta_{L,M}(K,Q,S)A_{Q,S}^{K}(t)\label{eq:BLM-tensor-AF}\end{aligned}$$

In both cases a set of geometric tensor terms are required, which are
fully defined in Appendix
[\[appendix:formalism\]](#appendix:formalism){reference-type="ref"
reference="appendix:formalism"}; these terms provide details of:

-   ${E_{P-R}(\hat{e};\mu_{0})}$: polarization geometry & coupling with
    the electric field.

-   $B_{L,S-R'}(l,l',m,m')$: geometric coupling of the partial waves
    into the $\beta_{L,M}$ terms (spherical tensors).

-   $\Lambda_{R'}(\mu,P,R')$: frame couplings and rotations.

-   $\Delta_{L,M}(K,Q,S)$: alignment frame coupling.

-   $A_{Q,S}^{K}(t)$: ensemble alignment described as a set of axis
    distribution moments (ADMs).

And $I_{l,m,\mu}^{p_{i}\mu_{i},p_{f}\mu_{f}}(\epsilon)$ are the (radial)
dipole ionization matrix elements, as a function of energy $\epsilon$.
These matrix elements are essentially identical to the simplified forms
$r_{k,l,m}$ defined in Eqn.
[\[eq:r-kllam\]](#eq:r-kllam){reference-type="ref"
reference="eq:r-kllam"}, except with additional indices to label
symmetry and polarization components defined by a set of partial-waves
$\{l,m\}$, for polarization component $\mu$ (denoting the photon angular
momentum components) and channels (symmetries) labelled by initial and
final state indexes ${p_{i}\mu_{i},p_{f}\mu_{f}}$. The notation here
follows that used by ePolyScat
[@Gianturco1994; @Lucchese1986; @Natalense1999], and these matrix
elements again represent the quantities to be obtained numerically from
data analysis, or from an [ePolyScat (or similar)
calculation](https://epsproc.readthedocs.io/en/latest/ePS_ePSproc_tutorial/ePS_tutorial_080520.html#Theoretical-background).

Note that, in this case as given, time-dependence arises purely from the
$A_{Q,S}^{K}(t)$ terms in the AF case, and the electric field term
currently describes only the photon angular momentum coupling, although
can in principle also describe time-dependent/shaped fields. Similarly,
a time-dependent initial state (e.g. a vibrational wavepacket) could
also describe a time-dependent MF case.

It should be emphasized, however, that the underlying physical
quantities are essentially identical in all the theoretical approaches,
with a set of coupled angular-momenta defining the geometrical part of
the photoionization problem, despite these differences in the details of
the theory and notation.

### Information content[\[sec:info-content\]]{#sec:info-content label="sec:info-content"}

As discussed in ref. [@hockett2018QMP2], the information content of a
single observable might be regarded as simply the number of contributing
$\beta_{L,M}$ parameters. In set notation:

$$M=\mathrm{n}\{\beta_{L,M}\}$$

where $M$ is the information content of the measurement, defined as
$\mathrm{n}\{...\}$ the cardinality (number of elements) of the set of
contributing parameters. A set of measurements, made for some
experimental variable $u$, will then have a total information content:

$$M_{u}=\sum_{u}\mathrm{n}\{\beta_{L,M}^{u}\}$$

In the case where a single measurement contains multiple $\beta_{L,M}$,
e.g. as a function of energy $\epsilon$ or time $t$, the information
content will naturally be larger:

$$\begin{aligned}
M_{u} & = & \sum_{u,k,t}\mathrm{n}\{\beta_{L,M}^{u}(\epsilon,t)\}\\
 & = & M_{u}\times M\end{aligned}$$

where the second line pertains if each measurement has the same native
information content, independent of $u$. It may be that the variable $k$
is continuous (e.g. photoelectron energy), but in practice it will
usually be discretized in some fashion by the measurement.

In terms of purely experimental methodologies, a larger $M_{u}$ clearly
defines a richer experimental measurement which explores more of the
total measurement space spanned by the full set of
$\{\beta_{L,M}^{u}(k,t)\}$. However, in this basic definition a larger
$M_{u}$ does not necessarily indicate a higher information content for
quantum retrieval applications. The reason for this is simply down to
the complexity of the problem (cf. Eqn.
[\[eqn:channel-fns\]](#eqn:channel-fns){reference-type="ref"
reference="eqn:channel-fns"}), in which many couplings define the
sensitivity of the observable to the underlying system properties of
interest. In this sense, more measurements, and larger $M$, may only add
redundancy, rather than new information.

A more complete accounting of information content would, therefore, also
include the channel couplings, i.e. sensitivity/dependence of the
observable to a given system property, in some manner. For the case of a
time-dependent measurement, arising from a rotational wavepacket, this
can be written as:

$$M_{u}=\mathrm{n}\{\varUpsilon_{L,M}^{u}(\epsilon,t)\}$$

In this case, each $(\epsilon,t)$ is treated as an independent
measurement with unique information content, although there may be
redundancy as a function of $t$ depending on the nature of the
rotational wavepacket and channel functions - this is explored further
in Sect.
[\[sec:bootstrapping-info-sensitivity\]](#sec:bootstrapping-info-sensitivity){reference-type="ref"
reference="sec:bootstrapping-info-sensitivity"}. (Note this is in
distinction to previously demonstrated cases where the time-dependence
was created from a shaped laser-field, and was integrated over in the
measurements, which provided a coherently-multiplexed case, see refs.
[@hockett2014CompletePhotoionizationExperiments; @hockett2015CompletePhotoionizationExperiments; @hockett2015CoherentControlPhotoelectron]
for details.)